In [1]:
# Import libraries
import requests
import json
import datetime
import pandas as pd
import time

#need information for the API
key = ""
end = datetime.date(2020, 7, 1)
start = datetime.date(2010, 6, 1)
url1 = "https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date="
url2 = "&end_date="
url3 = "&fq=organizations%3A(%22AMAZON.COM%20INC%22)&api-key="

In [2]:
#api only will return one month at a times this creates a list of all the months in the wanted time range
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m")]

In [3]:
#reformats the dates in the list so it is readable for the API
times = []
for line in months_in_range:
    temp = line[0]
    temp = temp.split('-')
    temp = ''.join(temp)
    times.append(temp)    

In [4]:
#puts together requestUrl and gets data from API
def Pulldata(Start, End):
    requestUrl = send = url1 + Start+ url2 + End + url3 + key
    request = requests.get(requestUrl)
    json_data = request.json()
    return json_data

In [5]:
#filters the API data the the wanted data
def FilterJson(data, ret):
    length = len(data['response']['docs'])
    for x in range(0,length):
        temp = {}
        temp['Date'] = data['response']['docs'][x]["pub_date"]
        temp['Headline'] = data['response']['docs'][x]["headline"]["main"]
        temp['Abstract'] = data['response']['docs'][x]["abstract"]
        keywords = data['response']['docs'][x]["keywords"]
        temp['Lead'] = data['response']['docs'][x]["lead_paragraph"]
        temp['Subjects'] = GetSubject(keywords)
        temp['News Desk'] = data['response']['docs'][x]["news_desk"]        
        temp['Source'] = data['response']['docs'][x]["source"]
        temp['Document Type'] = data['response']['docs'][x]["document_type"]
        temp['URL'] = data['response']['docs'][x]["web_url"]
        ret.append(temp)
    return ret
        

In [6]:
#Returns ony keywords that are relvant to the subject of the artical 
def GetSubject(keywords):
    loopfor = len(keywords)
    ret = []
    for x in range(0,loopfor):
        if keywords[x]["name"] == "subject":
            temp = keywords[x]["value"]
            ret.append(temp)
    return ret
        

In [7]:
#function that makes api request, filters out the data and returns it in a list
def Getnews(times):
    ret = []
    check = True
    loopFor = len(times) - 1
    for x in range(0,loopFor):       
        Start = times[x]
        End = times[x+1]
        json_data = Pulldata(Start, End)
        ret = FilterJson(json_data, ret)
        time.sleep(6)
    return ret
            
        
        

In [8]:
data = Getnews(times)

In [11]:
#create data frame 
outdf = pd.DataFrame(data, columns = ['Date', 'Headline','Abstract','Lead','Subjects','News Desk','Source',"document_type",'URL'])

In [13]:
outdf.to_csv('NytApiData.csv')